<a href="https://colab.research.google.com/github/VishnuGupta5883/appliedai/blob/master/Application_of_Bootstrap_samples_in_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Application of Bootstrap samples in Random Forest

In [0]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error

 <li> Load the boston house dataset </li>

In [0]:
boston = load_boston()
x=boston.data #independent variables
y=boston.target #target variable

In [0]:
x.shape

(506, 13)

### Task: 1
<font color='red'><b>Step 1 Creating samples: </b></font> Randomly create 30 samples from the whole boston data points.
<ol>
<li>Creating each sample: Consider any random 303(60% of 506) data points from whole data set and then replicate any 203 points from the sampled points</li>
<li>Ex: For better understanding of this procedure lets check this examples, assume we have 10 data points [1,2,3,4,5,6,7,8,9,10], first we take 6 data points randomly consider we have selected [4, 5, 7, 8, 9, 3] now we will replciate 4 points from [4, 5, 7, 8, 9, 3], consder they are [5, 8, 3,7] so our final sample will be [4, 5, 7, 8, 9, 3, 5, 8, 3,7]</li>
<li> we create 30 samples like this </li>
<li> Note that as a part of the Bagging when you are taking the random samples make sure each of the sample will have                different set of columns</li>
<li> Ex: assume we have 10 columns for the first sample we will select [3, 4, 5, 9, 1, 2] and for the second sample [7, 9, 1, 4, 5, 6, 2] and so on...</li>
<li> Make sure each sample will have atleast 3 feautres/columns/attributes</li>
</ol>

<font color='red'><b>Step 2 Building High Variance Models on each of the sample and finding train MSE value:</b></font> Build a DecisionTreeRegressor on each of the sample.
<ol><li>Build a regression trees on each of 30 samples.</li>
<li>computed the predicted values of each data point(506 data points) in your corpus.</li>
<li> predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{30}\sum_{k=1}^{30}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.</li>
<li>Now calculate the $MSE =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.</li>
</ol>

<font color='red'><b>Step 3 Calculating the OOB score :</b></font>
<ol>
<li>Computed the predicted values of each data point(506 data points) in your corpus.</li>
<li>Predicted house price of $i^{th}$ data point $y^{i}_{pred} =  \frac{1}{k}\sum_{\text{k= model which was buit on samples not included } x^{i}}(\text{predicted value of } x^{i} \text{ with } k^{th} \text{ model})$.</li>
<li>Now calculate the $OOB Score =  \frac{1}{506}\sum_{i=1}^{506}(y^{i} - y^{i}_{pred})^{2}$.</li>
</ol>

### Task: 2
<pre>
<font color='red'><b>Computing CI of OOB Score and Train MSE</b></font>
<ol>
<li> Repeat Task 1 for 35 times, and for each iteration store the Train MSE and OOB score </li>
<li> After this we will have 35 Train MSE values and 35 OOB scores </li>
<li> using these 35 values (assume like a sample) find the confidence intravels of MSE and OOB Score </li>
<li> you need to report CI of MSE and CI of OOB Score </li>
<li> Note: Refer the Central_Limit_theorem.ipynb to check how to find the confidence intravel</li>
</ol>
</pre>
### Task: 3
<pre>
<font color='red'><b>Given a single query point predict the price of house.</b></font>

<li>Consider xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60] Predict the house price for this point as mentioned in the step 2 of Task 1. </li>
</pre>

# Task-1

## Step-1 - Generating Bootstrap Sample

In [0]:
import random

def generate_sample(input_data,target_data):
  
  Selected_rows = np.random.choice(505,size=303,replace=False) # generate 303 random row
  Selected_rows = np.sort(Selected_rows)
  Selected_columns  = np.random.choice(13,size=13,replace=False)
  Selected_columns =  np.sort(Selected_columns)
  Repeated_rows = np.random.choice(Selected_rows,size=203,replace=False)# replicate 203 rows
  Repeated_rows = np.sort(Repeated_rows)
  sample_data = input_data[Selected_rows[:,None],Selected_columns]
  target_sample_data = target_data[Selected_rows]

  Replicated_sample_data = input_data[Repeated_rows[:,None],Selected_columns]
  Replicated_target_sample_data = target_data[Repeated_rows]

  final_sample_data = np.concatenate((sample_data,Replicated_sample_data))
  final_target_data = np.concatenate((target_sample_data, Replicated_target_sample_data))

  # generate oob rows
  oob_rows = np.array([i for i in range(0,506) if i not in Selected_rows])
  oob_data = input_data[oob_rows[:,None],Selected_columns]

  return final_sample_data,final_target_data,Selected_rows,Selected_columns,oob_data

list_input_data =[]
list_output_data =[]
list_selected_rows =[]
list_selected_columns =[]
list_oob_data =[]

for i in range(30):
  a,b,c,d,e = generate_sample(x,y)
  list_input_data.append(a)
  list_output_data.append(b)
  list_selected_rows.append(c)
  list_selected_columns.append(d)
  list_oob_data.append(e)



In [0]:
list_selected_columns[0].shape

(13,)

## Step-2 - Calculate MSE values

### Train Model

In [0]:
from sklearn.tree import DecisionTreeRegressor

def model_train_gen(input_data,output_data):
  models=[]

  for j in range(len(input_data)):

    regressor = DecisionTreeRegressor(max_depth = None)
    model = regressor.fit(input_data[j],output_data[j])
    models.append(model)
  return models



### MSE calc

In [0]:
def mse_score_gen(x,models,selected_columns):
  y_pred_for_mse=[]

  for i in x:
    y_pred =[]

    for j in range(len(models)):
      y_pred.append(models[j].predict(i[selected_columns[j]].reshape(1,-1)))
    y_pred_for_mse.append(np.median(np.asarray(y_pred)))

  mse = mean_squared_error(y,np.asarray(y_pred_for_mse))


  
  return mse



## Step -3- Calculate OOB score

In [0]:
def oob_score_gen(x,models,selected_columns,selected_rows):
  y_pred_for_oob=[]

  for i in range(len(x)):
    a=x[i]
    y_pred =[]

    for j in range(len(models)):
      if i not in selected_rows[j]:
        y_pred.append(models[j].predict(a[selected_columns[j]].reshape(1,-1)))
    y_pred_for_oob.append(np.median(np.asarray(y_pred)))

  oob = mean_squared_error(y,np.asarray(y_pred_for_oob))

  
  return oob



# Execution of MSE and OOB score

In [0]:
models = model_train_gen(list_input_data,list_output_data)
mse = mse_score_gen(x,models,list_selected_columns)
print(mse)
oob = oob_score_gen(x,models,list_selected_columns,list_selected_rows)
print(oob)

0.328498023715415
11.256506916996047


# Task-2 -Computing CI of OOB Score and Train MSE

In [0]:
MSE_arr =[]
OOB_arr =[]
list_input_data =[]
list_output_data =[]
list_selected_rows =[]
list_selected_columns =[]
list_oob_data =[]

for j in range(35):

  for i in range(30):
    o,p,q,r,s = generate_sample(x,y)
    list_input_data.append(o)
    list_output_data.append(p)
    list_selected_rows.append(q)
    list_selected_columns.append(r)
    list_oob_data.append(s)
  
  updated_models = model_train_gen(list_input_data,list_output_data)

  mse_val = mse_score_gen(x,updated_models, list_selected_columns)
  MSE_arr.append(mse_val)
  
  oob_val = oob_score_gen(x,updated_models,list_selected_columns,list_selected_rows)
  OOB_arr.append(oob_val)

print(len(MSE_arr))
print(len(OOB_arr))

35
35


In [0]:
MSE_arr

[0.21084486166007912,
 0.1510326086956522,
 0.14990118577075104,
 0.1505533596837945,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416,
 0.14958498023715416]

## MSE CI

In [0]:
np.percentile(MSE_arr, [2.5,  97.5])


array([0.14958498, 0.16000445])

## OOB CI

In [0]:
np.percentile(OOB_arr, [2.5,  97.5])

array([ 8.91964822, 10.75953977])

# Task-3 - Predict house price

In [0]:
import numpy as np
xq= [0.18,20.0,5.00,0.0,0.421,5.60,72.2,7.95,7.0,30.0,19.1,372.13,18.60]
xq= np.array(xq).reshape(1,-1)
xq.shape

(1, 13)

In [0]:
def pred_price(xq,models,selected_columns):
  y_pred_price=[]

  for i in xq:
    y_pred =[]

    for j in range(len(models)):
      y_pred.append(models[j].predict(i[selected_columns[j]].reshape(1,-1)))
    y_pred_price.append(np.median(np.asarray(y_pred)))
 
  return y_pred_price

house_price = pred_price(xq,models,list_selected_columns)
print(house_price)

[18.5]
